## 11.4 条件随机场的学习算法

$$
\begin{eqnarray}
P_{w}(y \mid x) &=& \frac{\exp (\color{red}{w} \cdot F(y, x))}{Z_{w}(x)} \\
Z_{w}(x) &=& \sum_{y}\exp (\color{red}{w} \cdot F(y, x))
\end{eqnarray}
$$

条件随机场的学习算法，就是估计权重向量$w=\left( w_{1}, w_{2}, \cdots, w_{K} \right)^{T}$。  <font color=red>形式上同最大熵模型</font>
  
   
条件随机场模型实际上是<font color=blue>定义在时序数据上的对数线性模型，</font>其学习方法包括MLE和正则化的MLE。具体的优化实现算法有改进的迭代尺度法IIS、GD以及拟牛顿法。

### 11.4.1 改进的迭代尺度法IIS

参见《LR和最大熵模型》

#### Step1 写出似然函数

已知训练数据，由此可知经验概率分布$\tilde{P}(X,Y)$，训练数据的对数似然函数为:  
  
  
$$
\begin{eqnarray}
L(w) = L_{\tilde{P}}(P_{w}) &=& \log \prod_{x,y}P_{w}(y|x)^{\tilde{P}(x,y)} = \sum_{x,y}\tilde{P}(x,y)\log P_{w}(y|x) \\
&=& \sum_{x,y} \left[ \tilde{P}(x,y)\sum_{k=1}^{K}w_{k}f_{k}(x,y)-\tilde{P}(x,y)\log Z_{w}(x) \right] \\
&=& \color{red}{???}\sum_{j=1}^{N}\sum_{k=1}^{K}w_{k}f_{k}(x_{j},y_{j}) - \sum_{j=1}^{N} \log Z_{w}(x_{j})
\end{eqnarray}
$$  

$\sum_{j=1}^{N}$: $N$个样本  

<font color=red>与《LR和最大熵模型》中式(6.26)不同，why?如何去掉$\tilde{P}(x,y)$的？</font>

#### Step2 写出更新方程

- 转移特征$t_{k}$的更新方程  


$$
\begin{eqnarray}
\sum_{x,y}\tilde{P}(x,y)\sum_{i=1}^{n+1}t_{k}(y_{i-1},y_{i},x,i) &&= E_\tilde{P}[t_{k}]\\
&&= \sum_{x,y}\tilde{P}(x)P(y|x)\sum_{i=1}^{n+1}t_{k}(y_{i-1},y_{i},x,i) \exp \left(\delta_{k}T(x,y)\right) \\
&& k=1,2,\cdots,K_{1} \tag{11.36}
\end{eqnarray}
$$

- 状态特征$s_{l}$的更新方程为:  
  
  
$$
\begin{eqnarray}
\sum_{x,y}\tilde{P}(x,y)\sum_{i=1}^{n}s_{l}(y_{i},x,i) &&= E_\tilde{P}[s_{l}]\\
&&= \sum_{x,y}\tilde{P}(x)P(y|x)\sum_{i=1}^{n}s_{l}(y_{i},x,i) \exp \left(\delta_{K_{1}+l}T(x,y)\right) \\
&& k=1,2,\cdots,K_{2} \tag{11.37}
\end{eqnarray}
$$

这里$T(x,y)$是在数据中$(x,y)$中出现所有特征数的总和:  
  
  
$$
T(x,y) = \sum_{k}f_{k}(y,x) = \sum_{k=1}^{K}\sum_{i=1}^{n+1}f_{k}\left(y_{i-1},y_{i},x,i\right)
$$  

$f_{k}(y,x)$是{0,1}的二值函数。

#### 算法: CRF学习的IIS法

输入: 特征函数$t_{1}, t_{2}, \cdots, t_{K_{1}}$，$s_{1}, s_{2}, \cdots, s_{K_{2}}$；经验分布$\tilde{P}(X,Y)$;(y|w)$;  
  

输出: 参数估计值$\hat{w}$；模型$P_{\hat{w}}$  
  
  
（1） 对所有$k \in \{1,2,\cdots,K\}$，取初值$w_{k}=0$;  
  
  
（2） 对每一$i \in \{1,2,\cdots,K\}$  
  
  
  （2.1）当$k=1,2,\cdots,K_{1}$时，令$\delta_{k}$是下面方程的解：
  $$
  \sum_{x} \tilde{P}(x) \sum_{y} P_{w}(y \mid x) \sum_{i=1}^{n+1}t_{k}(y_{i-1},y_{i},x,i) \exp \left(\delta_{k}T(x,y)\right) = E_{\tilde{P}}(t_{k}) 
  $$
  
$\quad \space \space \quad$当$k=K_{1}+l,l=1,2,\cdots,K_{2}$时，令$\delta_{K_{1}+l}$是下面方程的解： 
      
$$
      \sum_{x,y}\tilde{P}(x)P(y|x)\sum_{i=1}^{n}s_{l}(y_{i},x,i) \exp \left(\delta_{K_{1}+l}T(x,y)\right)= E_\tilde{P}[s_{l}]
$$
  
  
  （2.2）更新$w_{k} \leftarrow w_{k} + \delta_{k}$
  
  
（3）如果不是所有$w_{k}$都收敛，重复步骤(2)。

#### 算法S

在式(11.36)和(11.37)中，$T(x,y)$表示数据$(x,y)$中的特征总数，对不同的数据$(x,y)$取值可能不同。(<font color=red>$T(x,y)$是公式推导中构造不等式引入的变量，取值可不那么精确。</font>)为了处理这个问题，定义松弛特征:  
  
  
$$
s(x,y)=S - \sum_{i=1}^{n+1}\sum_{k=1}^{K}f_{k}\left(y_{i-1},y_{i},x,i\right) \tag{11.39}
$$

式中，$S$是一个常数。选择足够大的常数$S$使得对训练数据集的所有数据$(x,y)$，$s(x,y)\geq 0$成立。这是特征总数取$S$。

由式(11.36)，对于转移特征$t_{k}$，$\delta_{k}$的更新方程:  
  
  
$$
\delta_{k}=\frac{1}{S}\log \frac{E_{\tilde{P}}[t_{k}]}{E_{p}[t_{k}]} \tag{11.41}
$$

其中，  

  
$$
E_{P}[t_{k}] = \sum_{x}\tilde{P}(x)\sum_{i=1}^{n+1}\sum_{y_{i-1},y_{i}}t_{k}\left(y_{i-1},y_{i},x,i\right)\frac{\alpha_{i-1}^{T}(y_{i-1}|x)M_{i}(y_{i-1},y_{i}|x)\beta_{i}(y_{i}|x)}{Z(x)}
$$

同理，对于状态特征$s_{l}$，$\delta_{k}$的更新方程:  
  
  
$$
\delta_{K_{1}+l}=\frac{1}{S}\log \frac{E_{\tilde{P}}[s_{l}]}{E_{p}[s_{l}]} \tag{11.45}
$$

其中，  
  
  
$$
E_{P}[s_{l}] = \sum_{x}\tilde{P}(x)\sum_{i=1}^{n}\sum_{y_{i}}s_{l}\left(y_{i},x,i\right)\frac{\alpha_{i}^{T}(y_{i}|x)\beta_{i}(y_{i}|x)}{Z(x)}
$$

在算法$S$中需要使常数$S$取足够大，这样一来，每步迭代的增量向量会变大<font color=red>(不是变小吗？？)</font>，算法收敛会变慢。

#### 算法$T$

算法$T$对每个观测序列$x$计算其特征总数最大值$T(x)$:  
  
  
$$
T(x) = \max_{y}T(x,y)
$$

利用前向-后向递推公式，可以很容易地计算$T(x)=t$.<font color=red></font>


这时，转移特征参数的更新方程可以写成:  
  
  
$$
\begin{eqnarray}
E_{\tilde{P}}(t_{k}) &=& \sum_{x} \tilde{P}(x) \sum_{y} P_{w}(y \mid x) \sum_{i=1}^{n+1}t_{k}(y_{i-1},y_{i},x,i) \exp \left(\delta_{k}T(x)\right) \\
&=& \color{red}{??? }\sum_{x} \tilde{P}(x) \alpha_{k,t}\exp (\delta_{k}\cdot t) \\
&=& \color{red}{??? }\sum_{t=0}^{T_{max}}\alpha_{k,t}\beta_{k}^{t} \tag{11.47}
\end{eqnarray}
$$

这里，$\alpha_{k,t}$是特征$t_{k}$的期待值，$\delta_{k}=\log \beta_{k}$。$\beta_{k}$是多项式方程（11.47）唯一实根，可以用牛顿法求得，从而求得相关的$\delta_{k}$.

同样地，状态特征的参数更新方程可以写成:  

$$
\begin{eqnarray}
E_{\tilde{P}}(s_{l}) &=& \sum_{x} \tilde{P}(x) \sum_{y} P_{w}(y \mid x) \sum_{i=1}^{n}s_{l}(y_{i},x,i) \exp \left(\delta_{K_{1}+l}T(x)\right) \\
&=& \color{red}{??? }\sum_{x} \tilde{P}(x) b_{l,t}\exp (\delta_{k}\cdot t) \\
&=& \color{red}{??? }\sum_{t=0}^{T_{max}}b_{l,t}\gamma_{l}^{t} \tag{11.48}
\end{eqnarray}
$$


这里，$b_{l,t}$是特征$s_{l}$的期待值，$\delta_{l}=\log \gamma_{l}$。$\gamma_{l}$是多项式方程（11.48）唯一实根，可以用牛顿法求得.

### 11.4.2 拟牛顿法

条件随机场模型:  
  
  
$$
P_{w}(y|x)=\frac{\exp \left(\sum_{i=1}^{n}w_{i}f_{i}(x,y)\right)}{\sum_{y} \exp \left(\sum_{i=1}^{n}w_{i}f_{i}(x,y)\right)} \tag{11.49}
$$

目标函数:  


$$
\min_{w} \quad f(w)=\sum_{x}\tilde{P}(x)\log \sum_{y} \exp\left( \sum_{i=1}^{n}w_{i}f_{i}(x,y) \right) - \sum_{x,y}\tilde{P}(x,y)\sum_{i=1}^{n}w_{i}f_{i}(x,y) \tag{11.50}
$$

梯度函数:  


$$
g(w) = \sum_{x,y}\tilde{P}(x)P_{w}(y|x)f(x,y)-E_{\tilde{P}}(f) \tag{11.51}
$$

#### CRF学习的BFGS算法

输入: 特征函数$f_{1}, t_{2}, \cdots, f_{n}$；经验分布$\tilde{P}(X,Y)$;  
  

输出: 参数估计值$\hat{w}$；模型$P_{\hat{w}}(y|x)$  
  
  
(1) 选定初始点$w^{(0)}$，取$B_{0}$为正定对称矩阵，置$k=0$;  
  
  
(2) 计算梯度$g_{k}=g(w^{(k)})$. 若$g_{k}=0$，则停止计算；否则转（3）  


(3) 由$B_{k}p_{k}=-g_{k}$求出$p_{k}$  
  
  
(4) 一维搜索: 求$\lambda_{k}$使得  
$$
f\left( x^{(k)} + \lambda_{k}p_{k} \right) = \min_{\lambda \geq 0}f\left( x^{(k)} + \lambda p_{k} \right)
$$  


(5) 置$w^{(k+1)}=w^{(k)} + \lambda_{k}p_{k}$  
  
  
(6) 计算梯度$g_{k+1}=g(w^{(k+1)})$. 若$g_{k+1}=0$，则停止计算；否则，按下式求出$B_{k+1}$  
  
  
$$
B_{k+1}=B_{k}+\frac{y_{k} y_{k}^{\mathrm{T}}}{y_{k}^{\mathrm{T}} \delta_{k}}-\frac{B_{k} \delta_{k} \delta_{k}^{\mathrm{T}} B_{k}}{\delta_{k}^{\mathrm{T}} B_{k} \delta_{k}}
$$
其中，  
  
  
$$
y_{k} = g_{k+1}-g_{k}, \quad \delta_{k}=w^{(k+1)} - w^{(k)}
$$  


(7) 置$k=k+1$，转(3).

## 11.5 CRF的预测算法

**预测问题:**
  
 
给定条件随机场$p(Y|X)$，输入序列（观测序列$x$），求条件概率最大的输出序列（标记序列）$y^{*}$.用Viterbi算法求解。

$$
\begin{aligned}
y^{*} &=\arg \max _{y} P_{w}(y \mid x) \\
&=\arg \max _{y} \frac{\exp (w \cdot F(y, x))}{Z_{w}(x)} \\
&=\arg \max _{y} \exp (w \cdot F(y, x)) \\
&=\arg \max _{y}(w \cdot F(y, x))
\end{aligned}
$$

于是，CRF的预测问题称为求非规范化概率最大的最优路径问题  


$$
\max_{y}\left( w \cdot F(y,x)\right) \tag{11.52}
$$

其中  


$$
\begin{aligned}
w &=\left(w_{1}, w_{2}, \cdots, w_{K}\right)^{\mathrm{T}} \\
F(y, x) &=\left(f_{1}(y, x), f_{2}(y, x), \cdots, f_{K}(y, x)\right)^{\mathrm{T}} \\
f_{k}(y, x) &=\sum_{i=1}^{n} f_{k}\left(y_{i-1}, y_{i}, x, i\right), \quad k=1,2, \cdots, K
\end{aligned}
$$

为了求解最优路径，将式(11.52)写成如下形式:  


$$
\max_{y} \sum_{i=1}^{n}w_{i}\cdot F_{i}(y_{i-1}, y_{i},x) \tag{11.53}
$$

#### CRF的Viterbi算法

输入: 模型特征向量$F(y,x)$和权值向量$w$，观测序列$x=(x_{1}, x_{2}, \cdots, x_{n})$;  
  
  
输出: 最优路径$y^{*}$

(1) 初始化  
  
  
$$
\delta_{1}(j) = w\cdot F_{1}(y_{0}=start,y_{1}=j,x), j=1,2,\cdots,m
$$  


(2) 递推. 对$i=2,3,\cdots,n$  
  
  
$$
\delta_{i}(l) = \max_{1\leq j \leq m} \left\{ \delta_{i-1}(j) + w\cdot F_{i}(y_{i-1}=j,y_{i}=l,x) \right\}, \quad l=1,2,\cdots,m \\
\Psi_{i}(l) = \arg \max_{1\leq j \leq m} \left\{ \delta_{i-1}(j) + w\cdot F_{i}(y_{i-1}=j,y_{i}=l,x) \right\}, \quad l=1,2,\cdots,m
$$  


(3) 终止
  
  
  
$$
\max_{y}\left( w \cdot F(y,x)\right) = \max_{1\leq j \leq m}\delta_{n}(j) \\
y_{n}^{*} = \arg \max_{1\leq j \leq m}\delta_{n}(j)
$$ 


(4) 返回路径  
  
  
$$
y_{i}^{*} = \Psi_{i+1}(y_{i+1}^{*}), \quad i=n-1,n-2,\cdots,1
$$  

求得最优路径$y^{*}=(y_{1}^{*},y_{2}^{*},\cdots,y_{n}^{*})$.


In [11]:
F = [[[0, 0], [0, 0], [1, 1]], [[1, 1], [1, 0], [1, 1]],
     [[0, 1], [1, 1], [1, 1]]]

w = [[[0, 0], [0, 0], [1, 0.5]], [[0.6, 1], [1, 0], [0.8, 0.5]],
     [[0, 1], [1, 0.2], [0.8, 0.5]]]

In [117]:
import numpy as np

def viterbi(F, w, x):
    I = len(F) # 序列长度
    S = len(F[0][0]) # 状态个数
    psi = [[-1 for _ in range(S)] for _ in range(I)]   # 路径列表
    delta = [0 for _ in range(S)] # 时刻i各状态对应的非规范化概率
    y = [-1 for _ in range(I)]
    
    for i in range(I):
        delta_tmp = [0 for _ in range(S)]
        state = -1
        for l in range(S): # 当前时刻状态
            # 遍历前一时刻每个状态
            state = 0
            max_delta = delta[0] + w[i][0][l] * F[i][0][l] + w[i][-1][l] * F[i][-1][l]
            for j in range(1, S):
                tmp_delta = delta[j] + w[i][j][l] * F[i][j][l] + w[i][-1][l] * F[i][-1][l]
                if tmp_delta > max_delta:
                    state = j
                    max_delta = tmp_delta
            delta_tmp[l] = max_delta
            psi[i][l] = state
        delta = delta_tmp
    
    print(psi)
    
    # 回溯
    y[-1] = delta.index(max(delta))
    for idx in range(I-2, -1, -1):
        y[idx] = psi[idx+1][y[idx+1]]
      
    print(y)

In [118]:
viterbi(F,w,[0,0,0])

[[0, 0], [0, 0], [1, 0]]
[0, 1, 0]


## 11.6 HMM vs. CRF

相同点: 
1. 都是图概率模型；   


2. 都可用于序列标注问题

**异：**  
1. HMM是生成模型，CRF是判别模型； 
  
  
2. HMM是概率有向图，CRF是概率无向图；  


3. HMM模型存在**观测独立性假设**，在序列标注问题中，不能考虑词语与词语之间的上下文特征。而CRF可以自定义状态特征函数，不仅可以表达观测之间的依赖，还可表示当前观测与前后多个状态之间的复杂依赖。